# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Técnicas Avançadas para Captura e Tratamento de Dados</font>

## <font color="blue"> Web Scraping </font>

## <font color="blue">Solução dos Exercícios </font>

**Material Produzido por Luis Gustavo Nonato**<br>
**Cemeai - ICMC/USP São Carlos**

---

In [1]:
import requests as rq

### Exercício 1)
Considere a página do Wikipedia [https://en.wikipedia.org/robots.txt](https://en.wikipedia.org/robots.txt).
Faça uma requisição para pegar o conteúdo de tal página e:
1. verifique se a requisição foi bem sucedida
2. visualize o tipo de dado retornado e note que é um texto pleno (não é HTML nem JSON)
3. visualize o conteúdo

In [2]:
r = rq.get('https://en.wikipedia.org/robots.txt') # fazendo a requisição


print(r.status_code)  # verificando se a requisição foi bem sucedida

print(r.headers['Content-Type'])  # verificando o tipo de dado
                                  # retornado pelo request

print(r.text)  # verificando o conteúdo retornado

200
text/plain; charset=utf-8
﻿# robots.txt for http://www.wikipedia.org/ and friends
#
# Please note: There are a lot of pages on this site, and there are
# some misbehaved spiders out there that go _way_ too fast. If you're
# irresponsible, your access to the site may be blocked.
#

# Observed spamming large amounts of https://en.wikipedia.org/?curid=NNNNNN
# and ignoring 429 ratelimit responses, claims to respect robots:
# http://mj12bot.com/
User-agent: MJ12bot
Disallow: /

# advertising-related bots:
User-agent: Mediapartners-Google*
Disallow: /

# Wikipedia work bots:
User-agent: IsraBot
Disallow:

User-agent: Orthogaffe
Disallow:

# Crawlers that are kind enough to obey, but which we'd rather not have
# unless they're feeding search engines.
User-agent: UbiCrawler
Disallow: /

User-agent: DOC
Disallow: /

User-agent: Zao
Disallow: /

# Some bots are known to be trouble, particularly those designed to copy
# entire sites. Please obey robots.txt.
User-agent: sitecheck.internetseer

### Exercício 2)
Considere o site de dados abertos da cidade de São Paulo [http://dados.prefeitura.sp.gov.br/pt_PT/](http://dados.prefeitura.sp.gov.br/pt_PT/).

Faça uma requisição para pegar o conteúdo da página inicial e verifique se a requisição foi bem sucedida. Verifique que o tipo de dado retornado é um conteúdo HTML.

In [3]:
r = rq.get('http://dados.prefeitura.sp.gov.br/pt_PT/')

print(r.status_code)
print(r.headers)
print('\n\nTipo de conteudo:',r.headers['Content-Type'])

200
{'Date': 'Wed, 29 May 2024 01:18:46 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Connection': 'keep-alive', 'Cache-Control': 'private', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '7403'}


Tipo de conteudo: text/html; charset=utf-8


### Exercício 3)
Escreva um código para realizar um parsing no HTML obtido no exercício 2). Extraia o texto que descreve o portal e que está abaixo do título "SOBRE O PORTAL".

**DICA**: Utilize o browser para visualizar o "source" do html e encontre os marcadores que identificam o título "SOBRE O PORTAL".

In [4]:
from bs4 import BeautifulSoup

portal_soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
# a informação do portal esta dentro da marcação
# <div class="box">
about = portal_soup.find_all('div',{'class':'box'})

for a in about:
    p = a.find('p')
    if p:             # como existem várias marcações <div class="box">
                      # buscamos por aquela que contenha uma
                      # marcação interna tipo 'p' (paragrafo)
        print(p.text)


        O Portal de Dados Abertos da Prefeitura de São Paulo reúne conjuntos de dados de todas as secretarias, subprefeituras e empresas públicas municipais. Com os recursos deste Portal, é possível construir aplicativos, visualizações de dados e obter informações essenciais sobre as políticas públicas e a administração da cidade. 
		Saiba mais.



### Exercício 4)
Extraia todos os grupos de dados que estão disponíveis no portal [http://dados.prefeitura.sp.gov.br/pt_PT/](http://dados.prefeitura.sp.gov.br/pt_PT/), os quais estão indicados abaixo do título "CONHEÇA OS GRUPOS"). Imprima o nome do grupo e a URL da página do grupo.

**DICA**: Visualize o "source" do html e encontre os marcadores que identificam os grupos.

In [6]:
# a informação dos grupos esta dentro da marcação
# <li class="media-item home-media-item">
grupos = portal_soup('li',{'class':"media-item home-media-item"})

for g in grupos:
    name = g.find('h3').text
    url = g.find('a').get('href')
    print(name,'-->',url)

Cultura --> /pt_PT/group/cultura
Demografia --> /pt_PT/group/demografia
Direitos Humanos --> /pt_PT/group/direitos
Educação --> /pt_PT/group/educacao
Esporte e Lazer --> /pt_PT/group/esporte-e-lazer
Infraestrutura --> /pt_PT/group/infraestrutura
Meio Ambiente --> /pt_PT/group/meio-ambiente
Moradia --> /pt_PT/group/moradia
Negócios --> /pt_PT/group/negocios
Orçamento e Gestão --> /pt_PT/group/financas
Participação Social --> /pt_PT/group/participacao-social
Saúde e bem estar --> /pt_PT/group/saude
Segurança urbana --> /pt_PT/group/seguranca
Trabalho e renda --> /pt_PT/group/trabalho
Transportes e mobilidade --> /pt_PT/group/transporte


### Exercício 5)
A página ['http://dados.prefeitura.sp.gov.br/pt_PT/dataset'](http://dados.prefeitura.sp.gov.br/pt_PT/dataset) do portal possui uma barra de busca para que se realize consultas às bases existentes. O método <font color='blue'>get</font> pode realizar uma requisição onde parâmetros são enviandos para a barra de busca.
Por exemplo:
```python
rq.get('http://dados.prefeitura.sp.gov.br/pt_PT/dataset',
       params={'q','seguranca'})
```
envia a palavra 'seguranca' para a barra de busca, retornando um conteúdo HTML com o resultado da busca.

Faça uma requisição enviando como parâmetro de busca a palavra "saude". Faça um parsing no conteúdo retornado e extraia os nomes e as URLs dos repositórios retornados.
Por exemplo,

```html
Cadastro dos Estabelecimentos de Saúde

Este conjunto de dados contém informações sobre todos os equipamentos municipais de saúde, fruto de um esforço realizado pelo Departamento de Produção e Análise de Informação...
```
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="background-color:green">XLS</span>
<span style="background-color:orange">CSV</span>

corresponde a um dos repositórios listados no conteúdo retornado.
O nome do repositório é "Cadastro dos Estabelecimentos de Saúde" o qual está linkado à página [/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude](http://dados.prefeitura.sp.gov.br/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude).
Armaze o conteúdo extraído em um dicionário onde a chave é o nome do repositório e o valor é a URL correspondente.


In [7]:
import requests as rq

params = {"q":'saude'}
b = rq.get('http://dados.prefeitura.sp.gov.br/pt_PT/dataset',params=params)

In [8]:
from bs4 import BeautifulSoup

bases = BeautifulSoup(b.text, 'html.parser')

In [9]:
saude = {}
data_sets = bases('li',{'class':"dataset-item"})
for d in data_sets:
    nome = d.find('h3',{'class':"dataset-heading"}).text
    url = d.find('ul',{'class':"dataset-resources unstyled"}).find('a').get('href')
    saude[nome]=url

In [10]:
for k,v in saude.items():
    print(k,v)


Cadastro dos Estabelecimentos de Saúde
 /pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude

Folha de Pagamento - HSPM
 /pt_PT/dataset/folha-de-pagamento-hspm

Folha de Pagamento - AHMSP
 /pt_PT/dataset/folha-de-pagamento-ahmsp

Pedidos de informação protocolados à Prefeitura via sistema e-SIC
 /pt_PT/dataset/pedidos-de-informacao-protocolados-a-prefeitura-via-e-sic1

Relação de Servidores da Saúde do Município de São Paulo
 /pt_PT/dataset/relacao-de-servidores-da-saude-do-municipio-de-sao-paulo

Índice de Desenvolvimento Humano Municipal (IDHM)
 /pt_PT/dataset/indice-de-desenvolvimento-humano-municipal

Cadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de S...
 /pt_PT/dataset/cadastro-nacional-de-enderecos-para-fins-estatisticos-censo-2010

Auditorias e Notas Técnicas - CGM
 /pt_PT/dataset/auditorias-e-notas-tecnicas-cgm


### Exercício 6)
Cada uma das URLs obtidas no exercício 5) leva a um repositório contendo arquivos para download. Escreva um código para realizar uma requisição em todas as páginas apontadas pelas URLs obtidas no exercício 5). Armazene o resultado das requisições em uma lista chamada `repos`, ou seja, cada ítem da lista corresponde ao conteúdo retornado pela requisição feita a um repositório.

In [11]:
repos = []
for v in saude.values():
    url = 'http://dados.prefeitura.sp.gov.br'
    url += v
    print(url)
    r = rq.get(url)
    if r.status_code == 200:
        repos.append(r.text)

print(len(repos))

http://dados.prefeitura.sp.gov.br/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/folha-de-pagamento-hspm
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/folha-de-pagamento-ahmsp
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/pedidos-de-informacao-protocolados-a-prefeitura-via-e-sic1
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/relacao-de-servidores-da-saude-do-municipio-de-sao-paulo
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/indice-de-desenvolvimento-humano-municipal
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/cadastro-nacional-de-enderecos-para-fins-estatisticos-censo-2010
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/auditorias-e-notas-tecnicas-cgm
8


### Exercício 7)
Os conteúdos armazenados na lista `repos` são páginas HTML. Realize um parsing em cada um dos ítens da lista `repos` buscando pelo nome do repositório, a data de criação e a data da última atualização do repositório. Imprima tais informações.

In [12]:
reposP = []
for r in repos:
    reposP.append(BeautifulSoup(r, 'html.parser'))

for r in reposP:
    nome = r.find('h1').text # os títulos estão marcados pela primeira
                             # tag 'h1' do HTML

    # a data de criacao e ultima atualizacao
    # estao dentro da tag <section class='additional-info'>
    # e marcadas por 'tr', 'td' e 'th'
    s = r.find('section',{'class':"additional-info"})
    tr = s.find_all('tr')
    for t in tr:
        if 'Última Atualização' in  t.find('th').text:
            atualiz = t.find('td').text
        if 'Data de criação' in t.find('th').text:
            cria = t.find('td').text

    print(nome.strip())  # strip remove as quebras de linha e tabulacao
    print(3*' '+"Criação:",cria)
    print(3*' '+"Última Atualização:",atualiz)


Cadastro dos Estabelecimentos de Saúde
   Criação: 

    Dezembro 4, 2015, 21:50 (UTC)


   Última Atualização: 

    Dezembro 4, 2015, 22:12 (UTC)


Folha de Pagamento - HSPM
   Criação: 

    Dezembro 8, 2015, 18:54 (UTC)


   Última Atualização: 

    Maio 16, 2024, 12:23 (UTC)


Folha de Pagamento - AHMSP
   Criação: 

    Dezembro 8, 2015, 18:48 (UTC)


   Última Atualização: 

    Abril 5, 2022, 17:56 (UTC)


Pedidos de informação protocolados à Prefeitura via sistema e-SIC
   Criação: 

    Dezembro 4, 2015, 19:25 (UTC)


   Última Atualização: 

    Abril 6, 2023, 18:41 (UTC)


Relação de Servidores da Saúde do Município de São Paulo
   Criação: 

    Outubro 3, 2016, 16:23 (UTC)


   Última Atualização: 

    Outubro 3, 2016, 16:25 (UTC)


Índice de Desenvolvimento Humano Municipal (IDHM)
   Criação: 

    Dezembro 5, 2015, 15:37 (UTC)


   Última Atualização: 

    Dezembro 5, 2015, 15:58 (UTC)


Cadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de São 

### Exercício 8)
Para cada repositório da lista `repos` imprima a quantidade total de arquivos XLS, CSV, etc., que estão disponíveis para download.

In [13]:
# os arquivos para download são marcado pela tag <li class='resource-item'>
for r in reposP:
    item = r.find_all('li',{'class':"resource-item"})
    print(r.find('h1').text.strip())
    print(3*' '+'Número de arquivos para download:',len(item))

Cadastro dos Estabelecimentos de Saúde
   Número de arquivos para download: 24
Folha de Pagamento - HSPM
   Número de arquivos para download: 16
Folha de Pagamento - AHMSP
   Número de arquivos para download: 13
Pedidos de informação protocolados à Prefeitura via sistema e-SIC
   Número de arquivos para download: 156
Relação de Servidores da Saúde do Município de São Paulo
   Número de arquivos para download: 2
Índice de Desenvolvimento Humano Municipal (IDHM)
   Número de arquivos para download: 4
Cadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de São Paulo
   Número de arquivos para download: 8
Auditorias e Notas Técnicas - CGM
   Número de arquivos para download: 390


### Exercício 9)
Alguns grupos de dados possuem muitos repositórios. Escreva um código para imprimir os nomes de todos os repositórios do grupo "Orçamento e Gestão".

In [14]:
import requests as rq
from bs4 import BeautifulSoup

# url do grupo "Orçamento e Gestão"
url = 'http://dados.prefeitura.sp.gov.br/group/financas'

ofr = rq.get(url)
of = BeautifulSoup(ofr.text, 'html.parser')

# identificando o numero de paginas que devem ser precorridas
# esta informação está na tag
# <div class='pagination pagination-centered'>
pagestag = of.find('div', {'class':'pagination pagination-centered'})
npages = len(pagestag.find_all('a'))

for i in range(1,npages):
    ofr = rq.get(url,params={'page':str(i)}) # fazendo o request da
                                             # pagina i
    print(5*'--'+'pagina '+str(i)+'--'*5)
    if ofr.status_code != 200:
        break
    of = BeautifulSoup(ofr.text, 'html.parser')
    item = of('li',{'class':"dataset-item"})
    for p in item:
        print(p.find('a').text)

----------pagina 1----------
Folha de Pagamento - Theatro Municipal
Folha de Pagamento - SPTRANS
Folha de Pagamento - SPURBANISMO
Folha de Pagamento - SPTURIS
Folha de Pagamento - SP Parcerias
Folha de Pagamento - SPOBRAS
Folha de Pagamento - SPDA
Folha de Pagamento - SPCINE
Folha de Pagamento - PRODAM
Folha de Pagamento - IPREM
Folha de Pagamento - HSPM
Folha de Pagamento - FUNDAÇÃO PAULISTANA
Folha de Pagamento - COHAB
Folha de Pagamento - CET
Fluxo de Caixa - SPURBANISMO
Fluxo de Caixa - SPTURIS
Fluxo de Caixa - SPTRANS
Fluxo de Caixa - SPSEC
Fluxo de Caixa - SPPARCERIAS
Fluxo de Caixa - SPOBRAS
----------pagina 2----------
Fluxo de Caixa - SPDA
Fluxo de Caixa - SPCINE
Fluxo de Caixa - PRODAM
Fluxo de Caixa - COHAB
Fluxo de Caixa - CET
Histórico de Remuneração dos Servidores Ativos da Prefeitura de São Paulo
Relação de Servidores Ativos da Prefeitura de São Paulo
Demonstrativo da Dívida Fundada
Execução Orçamentária
Base de Compras e Licitações
Investimentos em Publicidade
Lei Orçam